# Advanced NumPy

## `numpy` internals

In [ ]:
import numpy as np
np.random.seed(2374)

In [ ]:
arr = np.random.randint(10, size=(8,8))

In [ ]:
arr.itemsize, arr.dtype

In [ ]:
arr

In [ ]:
# How to step through array memory?

arr.strides

In [ ]:
arr.strides[0] == arr.shape[1] * arr.itemsize

In [ ]:
# But what about views?

arr_view = arr[::2, 1:]

In [ ]:
arr

In [ ]:
arr_view

In [ ]:
arr.flags

In [ ]:
arr_view.flags

In [ ]:
# View always has base array
arr_view.base is arr

In [ ]:
arr_view.strides

In [ ]:
arr_view.strides[0] == arr_view.shape[1] * arr_view.itemsize

In [ ]:
np.byte_bounds(arr_view)[0] - np.byte_bounds(arr)[0]

In [ ]:
np.byte_bounds(arr_view)[1] - np.byte_bounds(arr)[1]

In [ ]:
arr.T.strides

In [ ]:
arr.T.base is arr

## Cache effects

In [ ]:
large_arr = np.random.randint(100, size=(1000000,))

In [ ]:
STEP = 8
larger_arr = np.random.randint(100, size=(1000000*STEP,))

In [ ]:
%timeit -n 100 -r 3 large_arr.sum()

In [ ]:
%timeit -n 100 -r 3 larger_arr[::STEP].sum()

In [ ]:
del large_arr, larger_arr

In [ ]:
large_arr = np.random.randint(100, size=(1000,1000))

In [ ]:
%timeit -n 50 -r 3 large_arr.sum(axis=1)

In [ ]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=1)

In [ ]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=0)

In [ ]:
large_arr.T.base is large_arr

## Views and copies

In [ ]:
%timeit -n 100 -r 3 large_arr_copy = large_arr.copy()

In [ ]:
%timeit -n 100 -r 3 large_arr + 1

In [ ]:
%timeit -n 100 -r 3 np.add(large_arr, 1)

In [ ]:
%timeit -n 100 -r 3 np.add(large_arr, 1, out=large_arr)

In [ ]:
np.add(large_arr, 1, out=large_arr)

### Beware!

In [ ]:
A = np.random.randint(10, size=(10,10))
B = np.random.randint(10, size=(10,))

In [ ]:
A

In [ ]:
B

In [ ]:
A+B

In [ ]:
np.add(A, B)

In [ ]:
np.add(A, B, out=A)

In [ ]:
np.add(A, B, out=B)

# Broadcasting

In [ ]:
arr_2d = np.random.randint(10, size=(10, 3))
arr_1d_1 = np.random.randint(10, size=(3, ))
arr_1d_2 = np.random.randint(10, size=(10, ))

In [ ]:
arr_2d

In [ ]:
arr_1d_1

In [ ]:
arr_1d_2

In [ ]:
(arr_2d + arr_1d_1) - arr_2d

In [ ]:
arr_2d + arr_1d_1

In [ ]:
arr_2d + arr_1d_2

In [ ]:
(arr_2d + np.expand_dims(arr_1d_2, axis=1)) - arr_2d

In [ ]:
arr_3d = np.random.randint(10, size=(7, 10, 3))

In [ ]:
arr_1d_1.shape

In [ ]:
(arr_3d + arr_1d_1) - arr_3d

In [ ]:
arr_3d.shape, arr_1d_2.shape

In [ ]:
arr_1d_2

In [ ]:
(arr_3d + np.expand_dims(arr_1d_2, axis=1)) - arr_3d

Broadcasting rules:
    
- All input arrays with ndim smaller than the input array of largest ndim, have 1’s prepended to their shapes.
- The size in each dimension of the output shape is the maximum of all the input sizes in that dimension.
- An input can be used in the calculation if its size in a particular dimension either matches the output size in that dimension, or has value exactly 1.
- If an input has a dimension size of 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension. In other words, the stepping machinery of the ufunc will simply not step along that dimension (the stride will be 0 for that dimension).

## General rules

### Avoid loops

In [ ]:
def square_loop(a):
    """Calculate square of an array in loop. We assume 1D array here."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i]*a[i]
    return result

In [ ]:
large_arr = np.random.randint(100, size=(100000,))

In [ ]:
%timeit -n 10 -r 3 square_loop(large_arr)

In [ ]:
%timeit -n 10 -r 3 np.square(large_arr)

### Use broadcasting

In [ ]:
def row_loop(a, b):
    """Calculate square of an array in loop. We assume 1D array here."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i] + b
    return result

In [ ]:
large_arr = np.random.randint(100, size=(1000,1000))
large_b = np.random.randint(100, size=(1000,))

In [ ]:
%timeit -n 10 -r 3 row_loop(large_arr, large_b)

In [ ]:
%timeit -n 10 -r 3 np.add(large_arr, large_b)

In [ ]:
%timeit -n 10 -r 3 np.add(large_arr, large_b, out=large_arr)

In [ ]:
np.add(large_arr, large_b)

In [ ]:
row_loop(large_arr, large_b)

In [ ]:
np.expand_dims(np.arange(10), axis=0) + np.expand_dims(np.arange(10), axis=-1)

### Beware!

In [ ]:
A = np.random.randint(10, size=(10,10))
B = np.random.randint(10, size=(10,))

In [ ]:
A

In [ ]:
B

In [ ]:
A+B

In [ ]:
np.add(A, B)

In [ ]:
np.add(A, B, out=A)

In [ ]:
np.add(A, B, out=B)

# Linear algebra basics

In [ ]:
v = np.random.randint(10, size=(3,))
m = np.random.randint(10, size=(5, 3))

In [ ]:
m

In [ ]:
v

## Dot products, determinants, traces

In [ ]:
np.dot(m, v)

In [ ]:
np.dot(m, v.reshape((3,-1)))

In [ ]:
np.dot(v, m.T)

In [ ]:
s = np.random.randint(10, size=(3,3))
s_inv = np.linalg.inv(s)

In [ ]:
s

In [ ]:
s_inv

In [ ]:
np.dot(s_inv, s)

In [ ]:
np.linalg.det(s), np.linalg.det(s_inv)

In [ ]:
np.trace(s), np.trace(s_inv)

## Eigenvalues, eigenvectors

In [ ]:
evals, evectors = np.linalg.eig(s)

In [ ]:
evals.sum()

In [ ]:
s_diagonal = np.diag(evals)

In [ ]:
s_diagonal

And now our matrix can be decomposed as:
    
$$
s = VEV{-1}
$$

where $E$ is a diagonal matrix (with eigenvalues on main diagonal), and $V$ is a matrix where columns are eigenvectors.

In [ ]:
np.dot(evectors, np.dot(s_diagonal, np.linalg.inv(evectors)))

In [ ]:
s